Traitement des données pour vérifier s'il en manque



In [ ]:
!pip install housing

Importation des librairies


In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tarfile
import os
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt




Extraction du/des fichiers


In [4]:
file_path_general = 'general_data.csv'
file_path_employee ='employee_survey_data.csv'
file_path_manager = 'manager_survey_data.csv'
file_path_in = 'in_time.csv'
file_path_out = 'out_time.csv'



directory= "/content/Data/"

# Load the dataset
general_data = pd.read_csv(directory + file_path_general)
employee_data = pd.read_csv(directory + file_path_employee)
managers_data = pd.read_csv(directory + file_path_manager)
in_data = pd.read_csv(directory + file_path_in)
out_data = pd.read_csv(directory + file_path_out)


in_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
out_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

#general_data.info()

#general_data.describe()


general_data.head()



,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [ ]:
employee_data.head()
managers_data.head()
in_data.head()
out_data.head()

,count
BusinessTravel,
Travel_Rarely,3129
Travel_Frequently,831
Non-Travel,450


On cherche des infos sur les types de données qu'on a dans in et out


In [5]:
in_data.info()

numerical_in_columns = in_data.select_dtypes(include=['float64', 'int64']).columns

for column in numerical_in_columns:
    print(f"\nColonne: {column}")
    print(in_data[column].head())
    print(f"Type de données: {in_data[column].dtype}")
    print(f"Valeurs uniques: {in_data[column].nunique()}")
    print(f"Moyenne: {in_data[column].mean():.2f}")
    print(f"Écart-type: {in_data[column].std():.2f}")
    print(f"Min: {in_data[column].min()}")
    print(f"Max: {in_data[column].max()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 262 entries, EmployeeID to 2015-12-31
dtypes: float64(12), int64(1), object(249)
memory usage: 8.8+ MB

Colonne: EmployeeID
0    1
1    2
2    3
3    4
4    5
Name: EmployeeID, dtype: int64
Type de données: int64
Valeurs uniques: 4410
Moyenne: 2205.50
Écart-type: 1273.20
Min: 1
Max: 4410

Colonne: 2015-01-01
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 2015-01-01, dtype: float64
Type de données: float64
Valeurs uniques: 0
Moyenne: nan
Écart-type: nan
Min: nan
Max: nan

Colonne: 2015-01-14
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 2015-01-14, dtype: float64
Type de données: float64
Valeurs uniques: 0
Moyenne: nan
Écart-type: nan
Min: nan
Max: nan

Colonne: 2015-01-26
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 2015-01-26, dtype: float64
Type de données: float64
Valeurs uniques: 0
Moyenne: nan
Écart-type: nan
Min: nan
Max: nan

Colonne: 2015-03-05
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 2015-0

Conversion In et Out


Fusion des dataframe par EmployeeID


In [ ]:
# Fusionner tous les DataFrames sur EmployeeID
merged_data = general_data.merge(employee_data, on='EmployeeID', how='outer') \
    .merge(managers_data, on='EmployeeID', how='outer') \
    .merge(in_data, on='EmployeeID', how='outer') \
    .merge(out_data, on='EmployeeID', how='outer')


merged_data.head()

In [ ]:


null_value = merged_data.isnull().sum()
null_value_manager = managers_data.isnull().sum()
null_value_employee = employee_data.isnull().sum()
null_value_in = in_data.isnull().sum()
null_value_out = out_data.isnull().sum()

print(null_value)


Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64


Remplissage des valeurs numériques manquantes par la valeure médianne


In [ ]:
import missingno as msno

# Visualize missing data
msno.matrix(merged_data)
plt.show()

# Impute missing values
# For numerical columns, use median imputation
numerical_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
merged_data[numerical_columns] = merged_data[numerical_columns].fillna(merged_data[numerical_columns].median())


In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

le=LabelEncoder()
ordinal=OrdinalEncoder()
#suppression des ligne avec valeurs manquante ? a check
general_data = general_data.dropna()

#on utilise l'encoder pour n'avoir que des valeurs numériques dans nos données

categorical_columns = ["Attrition", "BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "Over18"]

# for col in column_to_encode:
#     general_data[col]=le.fit_transform(general_data[col])
general_data['Attrition'] = le.fit_transform(general_data['Attrition'])
general_data[categorical_columns] = ordinal.fit_transform(general_data[categorical_columns])


general_data

NameError: name 'general_data' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Sélectionner les features pertinentes
features = [col for col in general_data.columns if col != 'Attrition' and col != 'EmployeeID']

X = general_data[features]
y = general_data['Attrition']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Régression logistique
log_reg = LogisticRegression(random_state=42, max_iter=5000)
log_reg.fit(X_train, y_train)

# Prédire et évaluer
y_pred_log = log_reg.predict(X_test)
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

# Importance des variables pour la régression logistique
importance = pd.DataFrame({'feature': X.columns, 'importance': abs(log_reg.coef_[0])})
importance = importance.sort_values('importance', ascending=False)
print("Top 10 features (Logistic Regression):")
print(importance.head(10))




In [ ]:

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Prédire et évaluer
y_pred_rf = rf.predict(X_test)
print("\nAccuracy (Random Forest):", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Importance des variables pour Random Forest
importance_rf = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_})
importance_rf = importance_rf.sort_values('importance', ascending=False)
print("\nTop 10 features (Random Forest):")
print(importance_rf.head(10))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculer les métriques pour les deux modèles
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
log_reg_scores = [
    accuracy_score(y_test, y_pred_log),
    precision_score(y_test, y_pred_log, average='weighted'),
    recall_score(y_test, y_pred_log, average='weighted'),
    f1_score(y_test, y_pred_log, average='weighted')
]
rf_scores = [
    accuracy_score(y_test, y_pred_rf),
    precision_score(y_test, y_pred_rf, average='weighted'),
    recall_score(y_test, y_pred_rf, average='weighted'),
    f1_score(y_test, y_pred_rf, average='weighted')
]

# Créer le graphique
x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, log_reg_scores, width, label='Logistic Regression')
rects2 = ax.bar(x + width/2, rf_scores, width, label='Random Forest')

ax.set_ylabel('Scores')
ax.set_title('Comparison of Logistic Regression and Random Forest')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

# Ajouter les valeurs sur les barres
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()